In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
from sympy.matrices import Matrix, eye
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.message import Message, SOMStoichiometry, SOMReaction

from scipy.linalg import lu, inv

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [22]:
# make sure to run #BIOMD 18
# #112 is very tricky to detect errors (no errors?!)
simple = load_file_from_curated_data(383)
for r in simple.reactions:
  if r.category != cn.REACTION_BOUNDARY:
    print(r.makeIdentifier(is_include_kinetics=False))

PGA_prod_Vc: RuBP + CO2 + 2.00 NADPH -> 2.00 PGA
PGA_prod_Vo: RuBP + CO2 + 2.00 NADPH -> 1.50 PGA
PGA_cons: PGA -> RuBP
NADPH_prod: NADP -> NADPH


In [29]:
simple.molecules[0]

RuBP

In [33]:
som = SOM([simple.molecules[0], simple.molecules[1]])

In [34]:
som

{CO2, RuBP}

In [23]:
m = Message(simple)

In [24]:
m.analyze()

True

In [25]:
nodes = list(m.nodes)
nodes

[{CO2}, {PGA, RuBP}, {NADP, NADPH}]

In [35]:
s_r = SOMReaction(reactants=[SOMStoichiometry(nodes[0], 1.0)],
                 products=[SOMStoichiometry(nodes[1], 3.0), SOMStoichiometry(nodes[2], 1.2)], 
                 reaction_label = 'none')

In [39]:
SOMStoichiometry(nodes[0], 1.0).identifier == "{CO2} *  1.00"

True

In [36]:
print(s_r)
print(s_r.category)

none: {CO2} -> 3.00 {PGA, RuBP} + 1.20 {NADP, NADPH}
reaction_1_n


In [8]:
som.identifier

'{FeII}'

In [5]:
s = StoichiometryMatrix(simple)
s.isConsistent()

True

In [8]:
m.nodes

NodeView(({citrulline}, {NO3}, {NO}, {NADPplus}, {FeII}, {FeII_star_O2}, {FeII_NO}, {O2}, {FeIII}, {NADPH}, {FeIII_NO}, {FeIII_star, FeII_O2, FeII_star}))

In [9]:
m.reduced_reactions

[ReactionSummary(label='r6', reactants=[FeII_star_O2 *  1.00], products=[citrulline *  1.00, FeIII_NO *  1.00], category='reaction_1_n'),
 ReactionSummary(label='r10', reactants=[FeII_NO *  1.00, O2 *  1.00], products=[NO3 *  1.00, FeIII *  1.00], category='reaction_n_n'),
 ReactionSummary(label='rF', reactants=[FeIII_NO *  1.00], products=[NO *  1.00, FeIII *  1.00], category='reaction_1_n'),
 ReactionSummary(label='r1', reactants=[FeIII *  1.00, NADPH *  1.00], products=[NADPplus *  1.00, FeII *  1.00], category='reaction_n_n'),
 ReactionSummary(label='r9', reactants=[FeII_NO *  1.00, FeIII *  1.00], products=[FeII *  1.00, FeIII_NO *  1.00], category='reaction_n_n'),
 ReactionSummary(label='r5', reactants=[O2 *  1.00, FeII_star *  1.00], products=[FeII_star_O2 *  1.00], category='reaction_n_1'),
 ReactionSummary(label='r2', reactants=[FeII_NO *  1.00, O2 *  1.00, FeIII *  1.00], products=[FeII_O2 *  1.00, FeIII_NO *  1.00], category='reaction_n_n'),
 ReactionSummary(label='r4', reac

In [8]:
m.stoichiometry_matrix

,r1,r2,r3,r4,r5,r6,rF,rG,r9,r10
FeIII,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
FeII,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
NO,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
FeII_star,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0
FeIII_star,0.0,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
NADPH,-1.0,0.0,0.0,-0.5,0.0,0.0,0.0,-0.5,0.0,0.0
FeII_NO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,-1.0
FeII_O2,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
citrulline,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
NADPplus,1.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0


In [9]:
m.rref_df.T

,FeIII,FeII,NO,FeII_star,FeIII_star,NADPH,FeII_NO,FeII_O2,citrulline,NADPplus,FeIII_NO,O2,NO3,FeII_star_O2
r1,-1.0,1.0,0.0,0.0,0.0,-1.0,0.0,0.00,0.0,1.0,0.00,0.0,0.00,0.0
r2,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0,0.00,-1.0,0.00,0.0
rF,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,1.00,0.0,1.0,-1.00,-1.0,0.00,0.0
r4,0.0,0.0,0.0,1.0,-1.0,-0.5,0.0,0.00,0.0,0.5,0.00,0.0,0.00,0.0
r5,0.0,0.0,0.0,0.0,-1.0,-0.5,0.0,0.00,0.0,0.5,0.00,-1.0,0.00,1.0
r9,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.00,0.0,-1.0,1.00,0.0,0.00,0.0
r10,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,1.00,0.0,0.0,1.00,-2.0,1.00,0.0
r3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.25,0.0,0.0,0.25,-0.5,-0.25,1.0
r6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,1.00,0.0,0.00,-1.0
rG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,-0.20,-0.6,0.20,0.2


In [11]:
len(m.nodes)

5

In [18]:
p, l, u = lu(m.stoichiometry_matrix.T)
l.shape[1]

7

In [15]:
print(u.shape[0])
print(u.shape[1])

7
9


In [8]:
print(inv(p))

[[ 1.  0.  0.  0.  0.  0. -0.]
 [ 0.  0.  0.  0.  0.  1. -0.]
 [ 0.  0.  0.  1.  0.  0. -0.]
 [ 0.  0.  0.  0.  1.  0. -0.]
 [ 0.  1.  0.  0.  0.  0. -0.]
 [ 0.  0.  1.  0.  0.  0. -0.]
 [ 0.  0.  0.  0.  0.  0.  1.]]


In [9]:
u

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0., -1.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  1.,  0., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [10]:
m.decomposeMatrix(m.stoichiometry_matrix).T

,R_smad_P_cyt,R_smad_P_nuc,R_smad_P_smad4_cyt,R_smad_P_smad4_nuc,R_smad_nuc,smad4_cyt,smad4_nuc,R_smad_cyt,Pi
reaction_1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
reaction_6,0.0,1.0,0.0,-1.0,0.0,0.0,1.0,0.0,0.0
reaction_2,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,-1.0,0.0
reaction_3,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,-1.0,0.0
reaction_5,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,1.0,0.0
reaction_4,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0
reaction_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
m.lower_inverse.dot(m.permuted_matrix.T)

,R_smad_P_cyt,R_smad_P_nuc,R_smad_P_smad4_cyt,R_smad_P_smad4_nuc,R_smad_nuc,smad4_cyt,smad4_nuc,R_smad_cyt,Pi
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
1,0.0,1.0,0.0,-1.0,0.0,0.0,1.0,0.0,0.0
2,-2.0,0.0,1.0,0.0,0.0,-1.0,0.0,1.0,0.0
3,1.0,0.0,-2.0,1.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0
6,0.0,-2.0,1.0,0.0,2.0,-1.0,0.0,-1.0,1.0


In [12]:
p, l, u = lu(m.stoichiometry_matrix.T)
p

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

In [13]:
l

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -0.,  1.,  0.],
       [ 0., -1.,  0., -1., -1., -1.,  1.]])

In [14]:
u

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0., -1.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  1.,  0., -1.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [15]:
l.dot(u)

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0., -1.,  0.,  0.,  1.,  0.,  0.],
       [-1.,  0.,  1.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.]])

In [16]:
# this is the matrix for operation (without transpose)
reduce_operation = inv(l).dot(inv(p))
reduce_operation

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1.]])

In [17]:
print(m.analyze())
m.rref_errors

AttributeError: 'Message' object has no attribute 'addMultiMultiReaction'

In [5]:
if m.rref_errors or m.type_one_errors or m.type_two_errors:
  print("at least one error")

at least one error


In [6]:
if [] or [] or []:
  print("none")

In [10]:
rref_df = m.decomposeMatrix(m.stoichiometry_matrix)
reactions = m.convertMatrixToReactions(simple, rref_df)

In [11]:
reactions

[ReactionSummary(label='reaction6', reactants=[XI *  1.00], products=[X *  1.00], category='reaction_1_1'),
 ReactionSummary(label='reaction5', reactants=[M *  1.00], products=[MI *  1.00], category='reaction_1_1'),
 ReactionSummary(label='reaction4', reactants=[], products=[], category='reaction_redundant'),
 ReactionSummary(label='reaction7', reactants=[], products=[], category='reaction_redundant')]

In [12]:
k = m.processUniUniReaction(reactions[0])

In [13]:
type(reactions[0].reactants[0].molecule)

SBMLLint.common.molecule.Molecule

In [14]:
str(reactions[0].label)

'reaction6'

In [15]:
m2 = MESGraph(simple)
m2.analyze()

Model analyzed...
No error detected.


''

In [16]:
s = StoichiometryMatrix(simple)
s.isConsistent()

True

In [17]:
list(m2.nodes)[1].reactions

{reaction4: MI -> M; cell * MI * V1 * pow(K1 + MI, -1)}